# Multi-Client Example

## Introduction

This example shows how to connect multiple `BeamNGpy` instances to the simulator and have them control different vehicles. For demonstration purposes, both instances are housed in the same Python process, but the same example could be used from an entirely different process or machine as well.

## Scenario

The scenario will be a rather simple one: there are two vehicles on the `tech_ground` map and one vehicle will randomly drive around, controlled by client A, with another vehicle, controlled by client B, mimicking A's behavior.

## Setup

Contrary to other examples, we will be using two instances of the `BeamNGpy` class representing two clients A and B. In our case, client A will create the scenario containing both vehicles, but client B will later connect and control one of the vehicles while A controls the other. The classes involved in this are:

* `BeamNGpy`: Two instances of this will be used to implement client A and B
* `Scenario`: Client A will create a scenario for both clients to play in
* `Vehicle`: Used to specify and control vehicles involved in the scenario
* `Electrics`: A sensor used to inspect interal systems of the vehicle

The code starts with importing the respective classes:

In [1]:
import random

from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics

Now we set up our first client who will create the scenario for both clients. Client A will also launch the simulator.

In [2]:
client_a = BeamNGpy("localhost", 64256)
client_a.open()

With the simulator running, we can start setting up our scenario. It will contain two vehicles, the one controlled by Client A being placed in front of the one later controlled by client B.

In [3]:
scenario = Scenario("tech_ground", "tag")
av_a = Vehicle("vehicleA", model="etk800")
av_b = Vehicle("vehicleB", model="etk800")
scenario.add_vehicle(av_a, pos=(0, -10, 0))
scenario.add_vehicle(av_b)
scenario.make(client_a)

## Running

The scenario is now made, meaning the required files have been generated and can be loaded in the simulator:

In [4]:
client_a.scenario.load(scenario)
client_a.scenario.start()

Now we will set up our second client and connect it to the running simulator. The client will first connect, then query the running scenario, and retrieve currently active vehicles. They will then find the vehicle meant for Client B and connect to it. Note that `client_b` is opened with the flag `launch=False`meaning it will not launch its own BeamNG.tech process.

In [5]:
client_b = BeamNGpy("localhost", 64256)
client_b.open(launch=False)
running_scenario = client_b.scenario.get_current()
print(running_scenario.name)
active_vehicles = client_b.vehicles.get_current()
bv_a = active_vehicles["vehicleA"]
bv_b = active_vehicles["vehicleB"]
# B attaches their own sensor to get the current controls of A
bv_a.sensors.attach("electrics", Electrics())
bv_a.connect(client_b)
bv_b.connect(client_b)

tag


Two clients are now connected to the running simulation and both vehicles. What follows is the main loop of the scenario, where Client A sends random steering inputs to their vehicle and Client B checks how A's vehicle is driving using the electrics sensor and sends the same inputs to their vehicle.

In [6]:
# Focus simulator on second vehicle because it's the more interesting one to see
av_b.focus()

for _ in range(20):
    # Client A sending control inputs to their vehicle connection
    av_a.control(throttle=random.random(), steering=(random.random() * 2 - 1))

    # Client B updating the electrics sensor of A's vehicle
    bv_a.sensors.poll()

    throttle = bv_a.sensors["electrics"]["throttle_input"]
    steering = bv_a.sensors["electrics"]["steering_input"]
    print(throttle, steering)
    bv_b.control(throttle=throttle, steering=steering)

    # Client A now advancing the simulation 60 steps
    client_a.control.step(60)

client_b.disconnect()
client_a.close()

0.9903552798732984 0.0561614481910334
0.37976397142866436 0.5152420182206601
0.8560437566064311 -0.3257185027727156
0.6696437525074509 0.5493611922590201
0.9156628563175278 -0.4430103829517375
0.9995778632528568 -0.28015915216392884
0.3641282530586958 0.27678702424977525
0.47662356655729277 -0.29412420724321964
0.8820322777695266 -0.1564247782230673
0.2007077839919995 -0.22352367524525876
0.7666620438170206 0.07661645227253353
0.5653868727133815 -0.06521475058326348
0.08736548992284598 -0.1529280373705123
0.8383866829879824 0.17128047529829377
0.9571386112800796 0.25183276732139354
0.18650957060048134 -0.1436863960145695
0.3787784585606455 -0.2766188613690152
0.9497251983984342 -0.26270946356273733
0.5483205382499797 0.21181323490602655
0.6323031607101598 0.10818393481871133
